# Tim Hortons Simulation Code

Note: This does not need to be an ipynb file, but I figure this might be simplest. 

In [13]:
############
# PACKAGES #
############

import numpy as np
import heapq # Event tree package


In [14]:
##############
# PARAMETERS #
##############

# TODO - CHOOSE SENSIBLE VALUES
MEAN_COFFEE_TIME = 0.1
MEAN_ESPRESSO_TIME = 0.1
MEAN_DONUT_TIME = 0.1
MEAN_PANINI_TIME = 0.1
MEAN_HASHBROWN_TIME = 0.1
MEAN_SANDWICH_TIME = 0.1

In [ ]:
###########
# CLASSES #
###########

# NOTE - We can change classes as needed

class Order:
    # Intialize an order object
    def __init__(self, order_id, type, num_customers, num_items, creation_time, expected_time):
        self.order_id = order_id           # Unique identifier attached to child food items as well
        self.num_customers = num_customers # Number of customers for seating, irrelevant to drive-thru
        self.num_items = num_items         # Number of customers for checking condition of order completion
        self.items_completed = 0           # Number of items completed, must equal num_items to complete order
        self.creation_time = creation_time # Time the order is first instantiated (once order enters customer service queue)
        self.expected_time = expected_time # (Only relevant to mobile orders)

        # Walk-in, pick-up, or drive-thru
        if (type.lower() == "walk-in") or (type.lower() == "drive-thru") or (type.lower() == "mobile"):
            self.order_type = type.lower()
        else:
            # Prevent bugs
            raise NameError("Invalid order type string ", type.lower())


class Food:
    # Initialize food object, each order has many corresponding food items
    def __init__(self, food_id, order_id, type, creation_time, expected_time):
        self.food_id = food_id             # Id for food item
        self.order_id = order_id           # Corresponding order
        self.food_type = type              # Type of food item (i.e. coffee, sandwich, etc.)
        self.creation_time = creation_time # Time the item is first instantiated (once order enters customer service queue)
        self.expected_time = expected_time # (Only relevant to mobile orders)

        # Type of food item (coffee, espresso, donut, panini, hashbrown, sandwich, etc.)
        if (type.lower() == "coffee") or (type.lower() == "espresso") or (type.lower() == "donut") or (type.lower() == "panini") or (type.lower() == "hashbrown") or (type.lower() == "sandwich"):
            self.food_type = type.lower()
        else:
            # Prevent bugs
            raise NameError("Invalid food type string ", type.lower())

        if type.lower() == "coffee":
            self.mean_service_time = MEAN_COFFEE_TIME
        elif type.lower() == "espresso":
            self.mean_service_time = MEAN_ESPRESSO_TIME
        elif type.lower() == "donut":
            self.mean_service_time = MEAN_DONUT_TIME
        elif type.lower() == "panini":
            self.mean_service_time = MEAN_PANINI_TIME
        elif type.lower() == "sandwich":
            self.mean_service_time = MEAN_SANDWICH_TIME
        elif type.lower() == "hashbrown":
            self.mean_service_time = MEAN_HASHBROWN_TIME
    

class Staff:
    # Create a staff worker
    def __init__(self, staff_id, type):
        self.staff_id = staff_id # Staff unique identifier
        self.staff_idle = True   # Bool for if the staff is working or not

        # Worker type (i.e. barista, cashier, kitchen, drive-thru window, etc.)
        if (type.lower() == "barista") or (type.lower() == "cashier") or (type.lower() == "kitchen") or (type.lower() == "window"):
            self.staff_type = type.lower()
        else:
            # Prevent bugs
            raise NameError("Invalid food type string ", type.lower())
    
    

class Equipment:
    # Create a piece of equipment
    def __init__(self, eq_id, type, num_slots):
        self.eq_id = eq_id         # Unique identifier for a specific piece of equipment
        self.eq_type = type        # The equipment type
        self.num_slots = num_slots # The total number of people who can work at a station/equipment
        self.used_slots = 0        # The number of equipment slots currently in use

        # Equipment type (i.e. panini-press, sandwich-station, coffee-maker, coffee-urn, etc.)
        if (type.lower() == "panini-press") or (type.lower() == "sandwich-station") or (type.lower() == "coffee-maker") or (type.lower() == "coffee-urn") or (type.lower() == "cash-register"):
            self.eq_type = type.lower()
        else:
            # Prevent bugs
            raise NameError("Invalid food type string ", type.lower())



class Event:
    def __init__(self, time, event_function, obj):
        self.time = time               # Time that the event must occur
        self.func = event_function # Function which triggers upon event
        self.obj = obj                 # Object assocaited with event (i.e. Food, Order, etc.)


In [16]:
###########################
# EVENT TREE INSTRUCTIONS #
###########################
##
# NOTE - The heapq package handles all the logic, and sorts by minimul value
##

event_queue = []

# To add an event object to the event tree in O(log(n)), do the following:
"""  
time = some_event_time
event_data = Event(some_stuff)
heapq.heappush(event_tree, (time, event_data))
"""

# Then to pop the event off of the event tree in O(log(n)), do the following:
"""
time, event = heapq.heappop(event_queue)
"""

# Or to peek at the next event in O(1), do the following:
"""
time, event = event_queue[0]
"""


'\ntime, event = event_queue[0]\n'

In [ ]:
class Kitchen:

    #####
    # X #
    #####
    def __init__(self, num_delays_required, num_cooks, num_baristas, num_panini, num_sandwich, num_coffee_maker, num_coffee_urn):
        # Staff
        self.cooks = [Staff(i, "kitchen") for i in range(num_cooks)]
        self.baristas = [Staff(i, "kitchen") for i in range(num_baristas)]

        # Equipment
        self.panini_press = Equipment(0, "panini-press", num_panini)
        self.sandwich_station = Equipment(0, "sandwich-station", num_sandwich)

        self.coffee_maker = Equipment(0, "coffee-maker", num_coffee_maker)
        self.coffee_urn = Equipment(0, "coffee-urn", num_coffee_urn)

        # Queues
        self.event_queue = []
        self.food_queue = []
        self.drink_queue = []
        self.output = []

        # State Variables
        self.sim_time = 0
        self.time_last_event = 0.0
        self.num_delays_required = num_delays_required
        self.num_delayed = 0

        # Ids
        self.next_food_id = 0
        self.next_order_id = 0

        # Interarrival times (Simulate Order arrivals)
        self.interarrival_time = 0.5
        self.add_next_food_arrival()
        self.add_next_drink_arrival()

        # Stats
        self.total_time_in_kitchen = 0
        self.area_in_food_queue = 0
        self.area_in_drink_queue = 0
    
    #####
    # X #
    #####
    def main(self):
        while self.num_delayed < self.num_delays_required:
            self.update_time_avg_stats()

            t, event = heapq.heappop(self.event_queue)
            self.sim_time = t
            # print("sim_time: " + str(self.sim_time))
            # print(len(self.food_queue))
            # print(event.func.__name__)
            event.func(*event.obj)

        # print(len(self.output))
        self.report()
    
    #####
    # X #
    #####
    """
    Stats
    """
    def update_time_avg_stats(self):
        time_since_last_event = self.sim_time - self.time_last_event

        self.area_in_food_queue += len(self.food_queue) * time_since_last_event
        self.area_in_drink_queue += len(self.drink_queue) * time_since_last_event

        self.time_last_event = self.sim_time

    #####
    # X #
    #####
    def report(self):
        print(f"total time in kitchen per item: {self.total_time_in_kitchen}")
        print(f"average time in kitchen per item: {self.total_time_in_kitchen / self.num_delays_required}")
        print(f"avg number in food_queue: {self.area_in_food_queue / self.sim_time}")
        print(f"avg number in drink_queue: {self.area_in_drink_queue / self.sim_time}")
        print(f"Total sim_time: {self.sim_time}")

    """
    Event Handlers
    """
    def handle_kitchen_arrival(self, food):

        self.add_next_food_arrival() # Simulate order arrivals

        # If corresponding equipment is unavailable
        equip = self.get_equipment(food)
        if not self.is_equipment_available(equip):
            self.food_queue.append(food)
            return

        # If any cooks are idle
        for cook in self.cooks:
            if cook.staff_idle:
                # assign cook and equipment slot
                cook.staff_idle = False
                equip.used_slots += 1

                # schedule departure
                time = self.sim_time + self.expon(food.mean_service_time)
                obj = (time, food, cook)
                event = Event(time, self.handle_kitchen_departure, obj)
                heapq.heappush(self.event_queue, (time, event))

                # increment number of customers delayed
                self.num_delayed += 1
                return
        
        # If equipment is available but no cooks, join queue
        self.food_queue.append(food)

    def handle_barista_arrival(self, drink):
        self.add_next_drink_arrival() # Simulate order arrivals

        # If corresponding equipment is unavailable
        equip = self.get_equipment(drink)
        if not self.is_equipment_available(equip):
            self.drink_queue.append(drink)
            return

        # If any baristas are idle
        for barista in self.baristas:
            if barista.staff_idle:
                # assign barista and equipment slot
                barista.staff_idle = False
                equip.used_slots += 1

                # schedule departure
                time = self.sim_time + self.expon(drink.mean_service_time)
                obj = (time, drink, barista)
                event = Event(time, self.handle_barista_departure, obj)
                heapq.heappush(self.event_queue, (time, event))

                # increment number of customers delayed
                self.num_delayed += 1
                return
        
        # If equipment is available but no cooks, join queue
        self.drink_queue.append(drink)

    def handle_kitchen_departure(self, time, food, cook):
        self.output.append(food)
        self.total_time_in_kitchen += time - food.creation_time

        equip = self.get_equipment(food)
        equip.used_slots -= 1

        # no food in queue
        if len(self.food_queue) == 0:
            cook.staff_idle = True
        else:
            # get next food that has an available equipment
            possible_food = self.get_possible_food(self.food_queue)

            if possible_food == []:
                cook.staff_idle = True
                return

            # Get food item with max priority
            # TODO - Implement advanced priority system
            next_food = max(possible_food, key=self.food_priority)
            
            # remove food from food_queue
            self.remove_from_queue(next_food, self.food_queue)

            # cook stays busy
            next_equip = self.get_equipment(next_food)
            next_equip.used_slots += 1

            # schedule departure
            time = self.sim_time + self.expon(next_food.mean_service_time)
            obj = (time, next_food, cook)
            event = Event(time, self.handle_kitchen_departure, obj)
            heapq.heappush(self.event_queue, (time, event))

            # increment number of customers delayed
            self.num_delayed += 1

    def handle_barista_departure(self, time, drink, barista):
        self.output.append(drink)
        self.total_time_in_kitchen += time - drink.creation_time

        equip = self.get_equipment(drink)
        equip.used_slots -= 1

        # no food in queue
        if len(self.drink_queue) == 0:
            barista.staff_idle = True
        else:
            # get next food that has an available equipment
            possible_drinks = self.get_possible_food(self.drink_queue)

            if possible_drinks == []:
                barista.staff_idle = True
                return

            # Get drink item with max priority
            # TODO - Implement advanced priority system
            next_drink = max(possible_drinks, key=self.food_priority)
            
            # remove food from drink_queue
            self.remove_from_queue(next_drink, self.drink_queue)

            # cook stays busy, new equipment is used
            next_equip = self.get_equipment(next_drink)
            next_equip.used_slots += 1

            # schedule departure
            time = self.sim_time + self.expon(next_drink.mean_service_time)
            obj = (time, next_drink, barista)
            event = Event(time, self.handle_barista_departure, obj)
            heapq.heappush(self.event_queue, (time, event))

            # increment number of customers delayed
            self.num_delayed += 1

    """
    Helper Functions
    """
    def expon(self, mean):
        return -mean * np.log(np.random.uniform(0, 1))

    # priority for food and beverage items
    def food_priority(self, food):
        return -food.creation_time

    def remove_from_queue(self, food, queue):
        for i in range(len(queue)):
            if food.food_id == queue[i].food_id:
                del queue[i]
                return
            
        print("Error: food not found in queue")

    # returns food in given array that have available equipment
    def get_possible_food(self, food_array):
        possible_food = []

        for food in food_array:
            if self.is_equipment_available(self.get_equipment(food)):
                possible_food.append(food)

        return possible_food

    def is_equipment_available(self, equip):
        return equip.num_slots > equip.used_slots
    
    def get_equipment(self, food):
        type = food.food_type
        if type == "panini":
            return self.panini_press
        if type == "sandwich":
            return self.sandwich_station
        if type == "coffee":
            return self.coffee_urn
        if type == "espresso":
            return self.coffee_maker
    
    def add_next_food_arrival(self):
        food_type = np.random.choice(["panini", "sandwich"])
        food = Food(self.next_food_id, self.next_order_id, food_type, -1, -1)
        time = self.sim_time + self.expon(self.interarrival_time)
        food.creation_time = time
        event = Event(time, self.handle_kitchen_arrival, (food,))
        heapq.heappush(self.event_queue, (time, event))

        self.next_food_id += 1
        self.next_order_id += 1

    def add_next_drink_arrival(self):
        food_type = np.random.choice(["coffee", "espresso"])
        food = Food(self.next_food_id, self.next_order_id, food_type, -1, -1)
        time = self.sim_time + self.expon(self.interarrival_time)
        food.creation_time = time
        event = Event(time, self.handle_barista_arrival, (food,))
        heapq.heappush(self.event_queue, (time, event))
        
        self.next_food_id += 1
        self.next_order_id += 1

In [18]:
MEAN_COFFEE_TIME = 1.0
MEAN_ESPRESSO_TIME = 1.0
MEAN_PANINI_TIME = 1.0
MEAN_SANDWICH_TIME = 1.0

num_delays_required = 50_000
num_cooks = 5
num_baristas = 5
num_panini = 3
num_sandwich = 3
num_coffee_maker = 3
num_coffee_urn = 3
kitchen = Kitchen(num_delays_required, num_cooks, num_baristas, num_panini, num_sandwich, num_coffee_maker, num_coffee_urn)
kitchen.main()

49996
total time in kitchen per item: 52587.62878242342
average time in kitchen per item: 1.0517525756484682
avg number in food_queue: 0.11865858972117847
avg number in drink_queue: 0.09981146277794035
Total sim_time: 12412.607777793071
